
HW 4.5 Clustering Tweet Dataset
Here you will use a different dataset consisting of word-frequency distributions for 1,000 Twitter users. These Twitter users use language in very different ways, and were classified by hand according to the criteria:
0: Human, where only basic human-human communication is observed.
1: Cyborg, where language is primarily borrowed from other sources (e.g., jobs listings, classifieds postings, advertisements, etc...).
2: Robot, where language is formulaically derived from unrelated sources (e.g., weather/seismology, police/fire event logs, etc...).
3: Spammer, where language is replicated to high multiplicity (e.g., celebrity obsessions, personal promotion, etc... )
Check out the preprints of recent research, which spawned this dataset:
http://arxiv.org/abs/1505.04342 http://arxiv.org/abs/1508.01843
The main data lie in the accompanying file:
topUsers_Apr-Jul_2014_1000-words.txt
and are of the form:
USERID,CODE,TOTAL,WORD1_COUNT,WORD2_COUNT,... . .
where
USERID = unique user identifier CODE = 0/1/2/3 class code TOTAL = sum of the word counts
Using this data, you will implement a 1000-dimensional K-means algorithm in MrJob on the users by their 1000-dimensional word stripes/vectors using several centroid initializations and values of K.
Note that each "point" is a user as represented by 1000 words, and that word-frequency distributions are generally heavy-tailed power-laws (often called Zipf distributions), and are very rare in the larger class of discrete, random distributions. For each user you will have to normalize by its "TOTAL" column. Try several parameterizations and initializations:
(A) K=4 uniform random centroid-distributions over the 1000 words (generate 1000 random numbers and normalize the vectors) (B) K=2 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution (C) K=4 perturbation-centroids, randomly perturbed from the aggregated (user-wide) distribution (D) K=4 "trained" centroids, determined by the sums across the classes. Use use the (row-normalized) class-level aggregates as 'trained' starting centroids (i.e., the training is already done for you!). Note that you do not have to compute the aggregated distribution or the class-aggregated distributions, which are rows in the auxiliary file:
topUsers_Apr-Jul_2014_1000-words_summaries.txt
Row 1: Words Row 2: Aggregated distribution across all classes Row 3-6 class-aggregated distributions for clases 0-3 For (A), we select 4 users randomly from a uniform distribution [1,...,1,000] For (B), (C), and (D) you will have to use data from the auxiliary file:
topUsers_Apr-Jul_2014_1000-words_summaries.txt
This file contains 5 special word-frequency distributions:
(1) The 1000-user-wide aggregate, which you will perturb for initializations in parts (B) and (C), and
(2-5) The 4 class-level aggregates for each of the user-type classes (0/1/2/3)
In parts (B) and (C), you will have to perturb the 1000-user aggregate (after initially normalizing by its sum, which is also provided). So if in (B) you want to create 2 perturbations of the aggregate, start with (1), normalize, and generate 1000 random numbers uniformly from the unit interval (0,1) twice (for two centroids), using:
from numpy import random numbers = random.sample(1000)
Take these 1000 numbers and add them (component-wise) to the 1000-user aggregate, and then renormalize to obtain one of your aggregate-perturbed initial centroids.
###################################################################################
##Geneate random initial centroids around the global aggregate
##Part (B) and (C) of this question
################################################################################### def startCentroidsBC(k): counter = 0 for line in open("topUsers_Apr-Jul_2014_1000-words_summaries.txt").readlines(): if counter == 2:
data = re.split(",",line) globalAggregate = [float(data[i+3])/float(data[2]) for i in range(1000)] counter += 1
#perturb the global aggregate for the four initializations    
centroids = []
for i in range(k):
    rndpoints = random.sample(1000)
    peturpoints = [rndpoints[n]/10+globalAggregate[n] for n in range(1000)]
    centroids.append(peturpoints)
    total = 0
    for j in range(len(centroids[i])):
        total += centroids[i][j]
    for j in range(len(centroids[i])):
        centroids[i][j] = centroids[i][j]/total
return centroids
—— For experiments A, B, C and D and iterate until a threshold (try 0.001) is reached. After convergence, print out a summary of the classes present in each cluster. In particular, report the composition as measured by the total portion of each class type (0-3) contained in each cluster, and discuss your findings and any differences in outcomes across parts A-D.
HW4.6 (OPTIONAL) Scaleable K-MEANS++
Over half a century old and showing no signs of aging, k-means remains one of the most popular data processing algorithms. As is well-known, a proper initialization of k-means is crucial for obtaining a good final solution. The recently proposed k-means++ initialization algorithm achieves this, obtaining an initial set of centers that is provably close to the optimum solution. A major downside of the k-means++ is its inherent sequential nature, which limits its applicability to massive data: one must make k passes over the data to find a good initial set of centers. The paper listed below shows how to drastically reduce the number of passes needed to obtain, in parallel, a good initialization. This is unlike prevailing efforts on parallelizing k-means that have mostly focused on the post-initialization phases of k-means. The proposed initialization algorithm k-means|| obtains a nearly optimal solution after a logarithmic number of passes; the paper also shows that in practice a constant number of passes suffices. Experimental evaluation on realworld large-scale data demonstrates that k-means|| outperforms k-means++ in both sequential and parallel settings.
Read the following paper entitled "Scaleable K-MEANS++" located at:
http://theory.stanford.edu/~sergei/papers/vldb12-kmpar.pdf
In MrJob, implement K-MEANS|| and compare with a random initializtion when used in conjunction with the kmeans algorithm as an initialization step for the 2D dataset generated using code in the following notebook:
https://www.dropbox.com/s/lbzwmyv0d8rocfq/MrJobKmeans.ipynb?dl=0
Plot the initialation centroids and the centroid trajectory as the K-MEANS|| algorithms iterates. Repeat this for a random initalization (i.e., pick a training vector at random for each inital centroid) of the kmeans algorithm. Comment on the trajectories of both algorithms. Report on the number passes over the training data, and time required to run both clustering algorithms. Also report the rand index score for both algorithms and comment on your findings.
4.6.1 Apply your implementation of K-MEANS|| to the dataset in HW 4.5 and compare to the a
random initalization (i.e., pick a training vector at random for each inital centroid)of the kmeans algorithm. Report on the number passes over the training data, and time required to run all clustering algorithms. Also report the rand index score for both algorithms and comment on your findings.
HW4.7 (OPTIONAL) Canopy Clustering
An alternative way to intialize the k-means algorithm is the canopy clustering. The canopy clustering algorithm is an unsupervised pre-clustering algorithm introduced by Andrew McCallum, Kamal Nigam and Lyle Ungar in 2000. It is often used as preprocessing step for the K-means algorithm or the Hierarchical clustering algorithm. It is intended to speed up clustering operations on large data sets, where using another algorithm directly may be impractical due to the size of the data set.
For more details on the Canopy Clustering algorithm see:
https://en.wikipedia.org/wiki/Canopy_clustering_algorithm
Plot the initialation centroids and the centroid trajectory as the Canopy Clustering based K-MEANS algorithm iterates. Repeat this for a random initalization (i.e., pick a training vector at random for each inital centroid) of the kmeans algorithm. Comment on the trajectories of both algorithms. Report on the number passes over the training data, and time required to run both clustering algorithms. Also report the rand index score for both algorithms and comment on your findings.
4.7.1 Apply your implementation Canopy Clustering based K-MEANS algorithm to the dataset in HW 4.5 and compare to the a
random initalization (i.e., pick a training vector at random for each inital centroid)of the kmeans algorithm. Report on the number passes over the training data, and time required to run both clustering algorithms. Also report the rand index score for both algorithms and comment on your findings.
=====================
END OF HOMEWORK

# HW4 DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  4
* **Date:**  6/7/16

**HW 4.0.**
What is MrJob? How is it different to Hadoop MapReduce? What are the mapper_init, mapper_final(), combiner_final(), reducer_final() methods? When are they called?  

**HW 4.1**
What is serialization in the context of MrJob or Hadoop? When it used in these frameworks? What is the default serialization mode for input and outputs for MrJob?

**HW 4.2:** Recall the Microsoft logfiles data from the async lecture. The logfiles are described are located at:
https://kdd.ics.uci.edu/databases/msweb/msweb.html http://archive.ics.uci.edu/ml/machine-learning-databases/anonymous/
This dataset records which areas (Vroots) of www.microsoft.com each user visited in a one-week timeframe in Feburary 1998.
Here, you must preprocess the data on a single node (i.e., not on a cluster of nodes) from the format:  
C,"10001",10001 #Visitor id 10001  
V,1000,1 #Visit by Visitor 10001 to page id 1000  
V,1001,1 #Visit by Visitor 10001 to page id 1001  
V,1002,1 #Visit by Visitor 10001 to page id 1002  
C,"10002",10002 #Visitor id 10001  
V  
Note: #denotes comments  
to the format:  
V,1000,1,C, 10001  
V,1001,1,C, 10001  
V,1002,1,C, 10001  

Write the python code to accomplish this.

In [2]:
'''
    This code reads the input file (infile), converts the data in it and writes the conversion
    to the output file (outfile).  The data is converted from writing customer information on
    a single line to writing customer information on the line corresponding its associated visit
    as per the instructions for HW4.2 above.
'''
infile = "anonymous-msweb.data"
outfile = "anonymous-msweb_converted.data"
with open(infile, 'r') as rf, open(outfile, 'w') as wf:
    for line in rf.readlines():
        # Split the lines on commas
        items = line.split(',')
        # If the line is a customer line, then save the customer ID for later use
        # Write the line to the output file
        if items[0] == 'C':
            cust_str = items[2]
            wf.write(line)
        # If the line is a visit line, then concatenate, the original line with the
        # current customer information (the current value of cust_str) and write
        # it to the output file
        elif items[0] == 'V':
            wf.write('%s,C,%s' % (line.strip(), cust_str))
        # All other lines write directly to the output file as is
        else:
            wf.write(line)

**HW 4.3:**  Find the 5 most frequently visited pages using MrJob from the output of 4.2 (i.e., transfromed log file).


In [2]:
%%writefile MostFrequentVisits.py

from mrjob.job import MRJob
#from mrjob.step import MRJobStep
from mrjob.step import MRStep
import csv

def csv_readline(line):
    """Given a sting CSV line, return a list of strings."""
    for row in csv.reader([line]):
        return row

#            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
#            'mapred.text.key.comparator.options': '-k2,2nr',

class MRMostFrequentVisits(MRJob):

    def mapper_count_visits(self, _, line):
        record = csv_readline(line)
        if record[0] == 'V':
            yield record[1], 1
    
    def reducer_sum_visits(self, page_id, counts):
        yield page_id, sum(counts)
    
    def reducer_sort_visits(self, page_id, counts):
        yield page_id, sum(counts)
    
    def steps(self):
        JOBCONF_STEP2 = {
            'stream.num.map.output.key.field': 2,
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k2,2nr',
            'mapred.reduce.tasks': '1',
        }
        return [
            MRStep(mapper=self.mapper_count_visits,   # STEP 1:  count the visits
                   reducer=self.reducer_sum_visits),
            MRStep(jobconf=JOBCONF_STEP2,
                    reducer=self.reducer_sort_visits)  # STEP 2:  sort the visits
        ]
    
if __name__ == '__main__':
    MRMostFrequentVisits.run()

Overwriting MostFrequentVisits.py


In [3]:
#!python MostFrequentVisits.py -r hadoop anonymous-msweb_converted.data
!python MostFrequentVisits.py anonymous-msweb_converted.data

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MostFrequentVisits.hadoop.20160603.223400.566995
Running step 1 of 2...
Running step 2 of 2...
Streaming final output from /tmp/MostFrequentVisits.hadoop.20160603.223400.566995/output...
"1000"	912
"1001"	4451
"1002"	749
"1003"	2968
"1004"	8463
"1005"	42
"1006"	135
"1007"	865
"1008"	10836
"1009"	4628
"1010"	698
"1011"	179
"1012"	44
"1013"	61
"1014"	728
"1015"	79
"1016"	287
"1017"	5108
"1018"	5330
"1019"	111
"1020"	1087
"1021"	380
"1022"	325
"1023"	191
"1024"	521
"1025"	2123
"1026"	3220
"1027"	507
"1028"	93
"1029"	132
"1030"	1115
"1031"	574
"1032"	1446
"1033"	26
"1034"	9383
"1035"	1791
"1036"	759
"1037"	1160
"1038"	1110
"1039"	345
"1040"	1506
"1041"	1500
"1042"	281
"1043"	224
"1044"	168
"1045"	474
"1046"	636
"1048"	210
"1049"	343
"1050"	106
"1051"	86
"1052"	842
"1053"	670
"1054"	338
"1055"	264
"1056"	276
"1057"	195
"1058"	672
"1059"	258
"1060"	391
"1061"	269
"1062"	141
"1063"	113
"1064"	324
"1065"	323
"106

In [ ]:
#from Mostused import MRMostUsedWord
#mr_job = MRMostUsedWord(args=['MostUsedWord.txt'])
#with mr_job.make_runner() as runner: 
#    runner.run()
#    # stream_output: get access of the output 
#    for line in runner.stream_output():
#        print mr_job.parse_output_line(line)
        
from MostFrequentVisits import MRMostFrequentVisits
mr_job = MRMostFrequentVisits (args=['anonymous-msweb_converted.data'])
with mr_job.make_runner() as runner:
    runner.run()
    # stream_output and print each line of the output
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

**HW 4.4:** Find the most frequent visitor of each page using MrJob and the output of 4.2 (i.e., transfromed log file). In this output please include the webpage URL, webpageID and Visitor ID.

In [5]:
%%writefile MostFrequentVisitors.py

from mrjob.job import MRJob
#from mrjob.step import MRJobStep
from mrjob.step import MRStep
import csv

def csv_readline(line):
    """Given a sting CSV line, return a list of strings."""
    for row in csv.reader([line]):
        return row

#            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
#            'mapred.text.key.comparator.options': '-k2,2nr',

class MRMostFrequentVisitors(MRJob):

    #SORT_VALUES = True
    
    def mapper_count_visits(self, _, line):
        record = csv_readline(line)
        if record[0] == 'V':
            yield (record[1],record[4]), 1
        
    def reducer_sum_visits(self, page_id, counts):
        s = sum(counts)
        if s > 1:
            yield page_id, sum(counts)
    
    def reducer_sort_visits(self, _, counts):
        yield max(counts)
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper_count_visits,   # STEP 1:  count the visits
                   reducer=self.reducer_sum_visits)
            #MRStep(reducer=self.reducer_sort_visits)  # STEP 2:  sort the visits
        ]
    
if __name__ == '__main__':
    MRMostFrequentVisitors.run()

Overwriting MostFrequentVisitors.py


In [6]:
!python MostFrequentVisitors.py anonymous-msweb_converted.data

No configs found; falling back on auto-configuration
Creating temp directory /tmp/MostFrequentVisitors.hadoop.20160603.223421.046240
Running step 1 of 1...
Streaming final output from /tmp/MostFrequentVisitors.hadoop.20160603.223421.046240/output...
Removing temp directory /tmp/MostFrequentVisitors.hadoop.20160603.223421.046240...


In [2]:
import numpy as np
size1 = size2 = size3 = 10000
samples1 = np.random.multivariate_normal([4, 0], [[1, 0],[0, 1]], size1)
data = samples1
samples2 = np.random.multivariate_normal([6, 6], [[1, 0],[0, 1]], size2)
data = np.append(data,samples2, axis=0)
samples3 = np.random.multivariate_normal([0, 4], [[1, 0],[0, 1]], size3)
data = np.append(data,samples3, axis=0)
# Randomlize data
data = data[np.random.permutation(size1+size2+size3),]
np.savetxt('Kmeandata.csv',data,delimiter = ",")

In [40]:
%%writefile Kmeans.py
from numpy import argmin, array, random
from mrjob.job import MRJob
from mrjob.step import MRStep
from itertools import chain
import os

#Calculate find the nearest centroid for data point 
def MinDist(datapoint, centroid_points):
    datapoint = array(datapoint)
    #print(datapoint)
    print(datapoint.shape)
    centroid_points = array(centroid_points)
    #print(centroid_points)
    print(centroid_points.shape)
    diff = datapoint - centroid_points 
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = argmin(list(diffsq.sum(axis = 1)))
    return minidx

#Check whether centroids converge
def stop_criterion(centroid_points_old, centroid_points_new,T):
    oldvalue = list(chain(*centroid_points_old))
    newvalue = list(chain(*centroid_points_new))
    Diff = [abs(x-y) for x, y in zip(oldvalue, newvalue)]
    Flag = True
    for i in Diff:
        if(i>T):
            Flag = False
            break
    return Flag

class MRKmeans(MRJob):
    centroid_points=[]
    k=4
    def steps(self):
        return [
            #MRStep(mapper_init = self.mapper_init, mapper=self.mapper,combiner = self.combiner,reducer=self.reducer)
            MRStep(mapper_init = self.mapper_init, mapper=self.mapper,reducer=self.reducer)
               ]
    #load centroids info from file
    def mapper_init(self):
        print "Current path:", os.path.dirname(os.path.realpath(__file__))
        
        self.centroid_points = [map(float,s.split('\n')[0].split(',')) for s in open("Centroids.txt").readlines()]
        #open('Centroids.txt', 'w').close()
        
        print "Centroids: ", self.centroid_points
        
    #load data and output the nearest centroid index and data point 
    def mapper(self, _, line):
        D = (map(float,line.split(',')))
        yield int(MinDist(D,self.centroid_points)), (D,1)
    
    #Combine sum of data points locally
    ##### DON'T CALL COMBINER RIGHT NOW
    def combiner(self, idx, inputdata):
        sumx = sumy = num = 0
        for x,y,n in inputdata:
            num = num + n
            sumx = sumx + x
            sumy = sumy + y
        yield idx,(sumx,sumy,num)
        
    #Aggregate sum for each cluster and then calculate the new centroids
    def reducer(self, idx, inputdata): 
        centroids = []
        num = [0]*self.k 
        # ?? see if you can parameterize the 1000
        for i in range(self.k):
            centroids.append([0]*1000)
        for D, n in inputdata:
            x = xy[0]
            y = xy[1]
            num[idx] = num[idx] + n
            centroids[idx][0] = centroids[idx][0] + x
            centroids[idx][1] = centroids[idx][1] + y
        centroids[idx][0] = centroids[idx][0]/num[idx]
        centroids[idx][1] = centroids[idx][1]/num[idx]
        
        print 'centroids updates:', centroids
        
        with open('Centroids.txt', 'w') as f:
            f.writelines(str(centroids[idx][0]) + ',' + str(centroids[idx][1]) + '\n')
        yield idx,(centroids[idx][0],centroids[idx][1])
      
if __name__ == '__main__':
    MRKmeans.run()

Overwriting Kmeans.py


In [46]:
%reload_ext autoreload
%autoreload 2
from numpy import random
from Kmeans import MRKmeans, stop_criterion
mr_job = MRKmeans(args=['topUsers_Apr-Jul_2014_1000-words_normalized_only_small.txt', '--file=Centroids.txt'])

#Geneate initial centroids FOR PART A
centroid_points = []
k = 4
with open('topUsers_Apr-Jul_2014_1000-words_normalized_small.txt', 'r') as f:
    lines = f.readlines()
    for i in range(k):
        #rint = random.randint(0, 999)
        rint = random.randint(0, 9)
        rline = lines[rint].strip().split(',')
        centroid_points.append([float(s) for s in rline[3:len(rline)]])
with open('Centroids.txt', 'w+') as f:
    f.writelines(','.join(str(j) for j in i) + '\n' for i in centroid_points)

# Update centroids iteratively
i = 0
#while(i):
while(i==0):
    # save previous centoids to check convergency
    centroid_points_old = centroid_points[:]
    print "iteration"+str(i)+":"
    with mr_job.make_runner() as runner: 
        runner.run()
        # stream_output: get access of the output 
        for line in runner.stream_output():
            key,value =  mr_job.parse_output_line(line)
            print key, value
            centroid_points[key] = value
    print "\n"
    i = i + 1
    if(stop_criterion(centroid_points_old,centroid_points,0.1)):
        break
print "Centroids\n"
print centroid_points

iteration0:
Current path: /tmp/Kmeans.hadoop.20160604.000844.685068/job_local_dir/0/mapper/0
Centroids:  [[0.134612, 0.000132, 7e-06, 0.0, 0.0, 0.0, 7e-06, 4.1e-05, 0.0, 0.0, 0.001078, 0.0, 0.000197, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.134619, 0.0, 0.0, 0.134619, 0.0, 0.000355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.135968, 0.0, 0.0, 0.0, 0.001608, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.004854, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01157, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.010228, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000643, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000425, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00091, 0.0, 0.0, 0.00158, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [10]:
# Create a normalized version of the data
infile = "topUsers_Apr-Jul_2014_1000-words.txt"
outfile = "topUsers_Apr-Jul_2014_1000-words_normalized.txt"
with open(infile, 'r') as rf, open(outfile, 'w') as wf:
    for line in rf.readlines():
        splt = line.strip().split(',')
        total = float(splt[2])
        wf.write('%s,%s,%s,' % (splt[0], splt[1], splt[2]))
        for i in range(3,len(splt)-1):
            wf.write('%f,' % (float(splt[i])/total))
        wf.write('%f' % (float(splt[len(splt)-1])/total))
        wf.write('\n')

# Create a normalized version of the data
infile = "topUsers_Apr-Jul_2014_1000-words.txt"
outfile = "topUsers_Apr-Jul_2014_1000-words_normalized_only.txt"
with open(infile, 'r') as rf, open(outfile, 'w') as wf:
    for line in rf.readlines():
        splt = line.strip().split(',')
        total = float(splt[2])
        for i in range(3,len(splt)-1):
            wf.write('%f,' % (float(splt[i])/total))
        wf.write('%f' % (float(splt[len(splt)-1])/total))
        wf.write('\n')

In [ ]:
###################################################################################
## Geneate random initial centroids around the global aggregate
## Part (B) and (C) of this question
###################################################################################
def startCentroidsBC(k):
    counter = 0
    for line in open("topUsers_Apr-Jul_2014_1000-words_summaries.txt").readlines():
        # Note correction from Kevin from Boulder
        if counter == 1:        
            data = re.split(",",line)
            globalAggregate = [float(data[i+3])/float(data[2]) for i in range(1000)]
        counter += 1
    ## perturb the global aggregate for the four initializations    
    centroids = []
    for i in range(k):
        rndpoints = random.sample(1000)
        peturpoints = [rndpoints[n]/10+globalAggregate[n] for n in range(1000)]
        centroids.append(peturpoints)
        total = 0
        for j in range(len(centroids[i])):
            total += centroids[i][j]
        for j in range(len(centroids[i])):
            centroids[i][j] = centroids[i][j]/total
    return centroids

In [22]:
#### TEST CELL ####
filename = "topUsers_Apr-Jul_2014_1000-words_normalized.txt"
with open(filename, 'r') as f:
    for line in f.readlines():
        s = line.split(',')
        #print(len(s))

In [19]:
#### TEST CELL ####
from numpy import argmin, array, random
from itertools import chain
import os
from Kmeans import MRKmeans, stop_criterion

def MinDist(datapoint, centroid_points):
    datapoint = array(datapoint)
    print(datapoint.shape)
    #print(datapoint)
    centroid_points = array(centroid_points)
    print(centroid_points.shape)
    #print(centroid_points)
    diff = datapoint - centroid_points
    print(diff)
    diffsq = diff*diff
    # Get the nearest centroid for each instance
    minidx = argmin(list(diffsq.sum(axis = 1)))
    return minidx

#Geneate initial centroids FOR PART A
centroid_points = []
k = 4
with open('topUsers_Apr-Jul_2014_1000-words_normalized.txt', 'r') as f:
    lines = f.readlines()
    for i in range(k):
        rint = random.randint(0, 999)
        rline = lines[rint].strip().split(',')
        centroid_points.append([float(s) for s in rline[3:len(rline)]])
    rline = lines[0].strip().split(',')
    datapoint = [float(s) for s in rline[3:len(rline)]]

minidx = MinDist(datapoint, centroid_points)
print(minidx)

(1000,)
(4, 1000)
[[ -4.82270000e-02  -1.43480000e-02   3.33740000e-02 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  2.67210000e-02  -4.06280000e-02  -7.14600000e-03 ...,   0.00000000e+00
   -8.30000000e-05  -8.30000000e-05]
 [  4.30300000e-02  -5.04220000e-02   8.33900000e-03 ...,   0.00000000e+00
   -6.70000000e-04  -1.51000000e-04]
 [ -8.89080000e-02  -4.29300000e-03   3.34010000e-02 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]
1
